In [257]:
!pip install thefuzz

In [258]:
import pandas as pd

In [259]:
df = pd.read_csv(r"C:\Users\Utilizador\OneDrive\Documentos\NovaIms\MachineLearning\Project_data\test.csv")
train = pd.read_csv(r"C:\Users\Utilizador\OneDrive\Documentos\NovaIms\MachineLearning\Project_data\train.csv")

## Data Preparation

### carID

In [262]:
df_concat = pd.concat([df, train], ignore_index=True)

duplicates = df_concat.duplicated().any()
print(duplicates)

df_concat[df_concat.duplicated(['carID'])]


False


,carID,Brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize,paintQuality%,previousOwners,hasDamage,price


<b>An attribute that contains an identifier for each car.
* The carID variable is 100% distinct across all records, hence we can use it as an index.

In [263]:
df = df.set_index(["carID"])
train = train.set_index(["carID"])

### Brand

In [264]:
print(train["Brand"].unique())

['VW' 'Toyota' 'Audi' 'Ford' 'BMW' 'Skoda' 'Opel' 'Mercedes' 'FOR'
 'mercedes' 'Hyundai' 'w' 'ord' 'MW' 'bmw' nan 'yundai' 'BM' 'Toyot' 'udi'
 'Ope' 'AUDI' 'V' 'opel' 'pel' 'For' 'pe' 'Mercede' 'audi' 'MERCEDES'
 'OPEL' 'koda' 'FORD' 'Hyunda' 'W' 'Aud' 'vw' 'hyundai' 'skoda' 'ford'
 'TOYOTA' 'ercedes' 'oyota' 'toyota' 'SKODA' 'Skod' 'HYUNDAI' 'kod' 'v'
 'for' 'SKOD' 'aud' 'KODA' 'PEL' 'yunda' 'or' 'UDI' 'OYOTA' 'HYUNDA' 'mw'
 'OPE' 'mercede' 'ERCEDES' 'ercede' 'TOYOT' 'MERCEDE' 'ORD' 'ud' 'ope'
 'AUD' 'hyunda' 'skod' 'toyot']


In [ ]:
#isto é o csv que é criado com recurso a api. Não vale a pena por a correr porque demora muito. Mas o ficheiro vai com o nome cars_api_request
#depois tem tudo de ser revisto porque esta bue gpt e com lixo
brand_model_dic = pd.read_csv(r"C:\Users\Utilizador\OneDrive\Documentos\NovaIms\MachineLearning\brands_models.csv")

### Model

In [267]:
print(df["model"].value_counts())

model
 Focus        2721
 C Class      2037
 Fiesta       1813
 Golf         1317
 Corsa         942
              ... 
YARIS            1
 I3              1
yaris            1
Combo Life       1
 GOLF SV         1
Name: count, Length: 593, dtype: int64


In [268]:
print(brand_model_dic["model"].value_counts())

model
7                  19
3                  11
5                  11
S5                  8
S7                  8
                   ..
Grand Lion 1949     1
Grand Lion 1986     1
Landmark            1
Little Tiger        1
Terralord           1
Name: count, Length: 5106, dtype: int64


Fazer fuzzmatching com os modelos sabendo que em caso de duvida podemos olhar para a marca. Mas so preenchemos/alteramos quando temos 0.95 ou mais de certeza
Se o modelo for unico, ignorar o valor de brand e meter a correta.
Se nao for unico, vamos fazer fuzzmathcing com brand a ver qual é que se adapta melhor dentro do universo de marcas que têm esse modelo.
Se houver model vazio, mas houve brand. Fazer fuzzy mathcing para a brand, escolher brand e depois dentro dessa brand escolher o modelo mais comun.
Se não houve brand nem model. Escolher a combinação modelo/brand mais comum e prencher com isso



In [ ]:
#aqui faço uma normalização basica primeiro par isto nao se passar so porque tem maiusculas e espaços e quês
train["Brand"] = train["Brand"].str.lower().str.strip()
train["model"] = train["model"].str.lower().str.strip()
brand_model_dic["brand"] = brand_model_dic["brand"].str.lower().str.strip()
brand_model_dic["model"] = brand_model_dic["model"].str.lower().str.strip()

In [271]:
# Dicionário de sinónimos relevantes (sem duplicar maiúsculas/minúsculas)
brand_aliases = {
    "vw": "volkswagen",
    "mb": "mercedes-benz",
    "mercedes benz": "mercedes-benz",
    "alfa": "alfa romeo",
    "alfa-romeo": "alfa romeo",
    "lr": "land rover",
    "range rover": "land rover",
    "vauxhall": "opel",
    "ds": "ds automobiles",
}

# Adiciona as equivalências ao teu brand_model_dic (já normalizado)
alias_rows = [{"brand": canonical, "model": ""} for canonical in set(brand_aliases.values())]
alias_rows += [{"brand": alias, "model": ""} for alias in brand_aliases.keys()]
brand_alias_df = pd.DataFrame(alias_rows)

brand_model_dic = pd.concat([brand_model_dic, brand_alias_df], ignore_index=True)
brand_model_dic.drop_duplicates(subset=["brand", "model"], inplace=True)
brand_model_dic.reset_index(drop=True, inplace=True)

In [272]:
train["Brand"] = (
    train["Brand"]
      .astype(str)
      .str.strip()
      .str.lower()           # já normalizas, mas fica aqui para ser à prova de bala
      .replace(brand_aliases)  # mapeia alias -> canónico (vw->volkswagen, mb->mercedes-benz, etc.)
)

In [274]:
print(train["Brand"].unique())

['volkswagen' 'toyota' 'audi' 'ford' 'bmw' 'skoda' 'opel' 'mercedes' 'for'
 'hyundai' 'w' 'ord' 'mw' 'nan' 'yundai' 'bm' 'toyot' 'udi' 'ope' 'v'
 'pel' 'pe' 'mercede' 'koda' 'hyunda' 'aud' 'ercedes' 'oyota' 'skod' 'kod'
 'yunda' 'or' 'ercede' 'ud']


In [ ]:
#tenho de rever esta função toda porque primeiro tá bue gpt. Depois esta a dar erros, mas ja da para ter uma ideia.
#em vez de substituir as colunas estou a criar novas para conseguir comparar as alterações
import re
import pandas as pd
from rapidfuzz import process, fuzz

# ---------------- utils ----------------
try:
    def fuzzy_best(query, choices):
        """Fuzzy sobre a lista ORIGINAL 'choices'. Devolve (string_escolhida, score[0..1], idx)."""
        if not choices:
            return None, 0.0, None
        q = str(query).strip().lower()
        choices_norm = [str(c).strip().lower() for c in choices]
        c_wr = process.extractOne(q, choices_norm, scorer=fuzz.WRatio)
        c_ts = process.extractOne(q, choices_norm, scorer=fuzz.token_set_ratio)
        opts = [c for c in (c_wr, c_ts) if c]
        if not opts:
            return None, 0.0, None
        best = max(opts, key=lambda c: c[1])
        idx = best[2]; score = best[1] / 100.0
        return choices[idx], score, idx
except Exception:
    from difflib import SequenceMatcher
    def _sim(a,b): return SequenceMatcher(None, a, b).ratio()
    def fuzzy_best(query, choices):
        if not choices:
            return None, 0.0, None
        q = str(query).strip().lower()
        best_item, best_sc, best_idx = None, 0.0, None
        for i,c in enumerate(choices):
            sc = _sim(q, str(c).strip().lower())
            if sc > best_sc: best_item, best_sc, best_idx = c, sc, i
        return best_item, best_sc, best_idx

NULL_TOKENS = {"", "nan", "none", "null", "n/a", "na", "-"}

def norm_text(s: str) -> str:
    return str(s).strip().lower()

def nullify(x) -> str:
    s = "" if x is None else str(x).strip()
    return "" if s.lower() in NULL_TOKENS else s

def norm_key(s: str) -> str:
    """Chave alfanumérica (remove espaços/pontuação): 'a-class'≡'aclass', 'up!'≡'up'."""
    return re.sub(r"[^a-z0-9]+", "", norm_text(s))

# ----------- aliases de brand (corrigem typos/abreviações) -----------
BRAND_ALIASES = {
    "vauxhall": "opel",
    "for": "ford", "ord": "ford",
    "mw": "bmw", "bm": "bmw",
    "mercede": "mercedes-benz", "ercedes": "mercedes-benz", "mercedes": "mercedes-benz",
    "aud": "audi", "udi": "audi",
    "yundai": "hyundai",
    "skod": "skoda",
    "toyot": "toyota",
    "ope": "opel", "pel": "opel",
    # ativa se fizer sentido:
    # "w": "volkswagen", "v": "volkswagen",
}

# ----------- normalizações específicas por brand (model) -----------
def brand_model_normalize(brand_can: str, model_raw: str) -> str:
    m = norm_text(model_raw)
    b = norm_text(brand_can)

    # Mercedes "classes": usar hífen; SUVs GL* idem
    m = re.sub(r"^([abcegs])\s*class$", r"\1-class", m)
    m = re.sub(r"^(gla|glb|glc|gle|gls)\s*class$", lambda mo: f"{mo.group(1)}-class", m)

    # VW up → 'up!' (a chave ignora '!')
    if b in {"volkswagen", "vw"}:
        if m in {"u", "up", "up !", "up!", "up!!"}:
            m = "up!"

    # Opel: Viva = Karl; zafira toure -> tourer
    if b == "opel":
        if m == "viva": m = "karl"
        m = m.replace("zafira toure", "zafira tourer")

    return m

# ---------------- pipeline principal ----------------
def clean(
    train: pd.DataFrame,
    brands_models: pd.DataFrame,
    thr_brand: float = 0.93,          # fuzzy para brand
    thr_model_in_brand: float = 0.93,  # fuzzy do model DENTRO da brand (primeiro)
    thr_model_unique: float = 0.97     # fuzzy global p/ assumir model único quando brand vazia
) -> pd.DataFrame:

    # MASTER
    bm = brands_models.copy()
    bm["brand"] = bm["brand"].map(nullify)
    bm["model"] = bm["model"].map(nullify)
    bm = bm[(bm["brand"]!="") & (bm["model"]!="")]

    brands = sorted(bm["brand"].unique().tolist())
    models  = sorted(bm["model"].unique().tolist())

    brand_to_models = (
        bm.groupby("brand")["model"].apply(lambda s: list(dict.fromkeys(s.tolist()))).to_dict()
    )
    # mapas para igualdade forte por chave
    brand_to_models_keymap = {b: {norm_key(m): m for m in mods} for b,mods in brand_to_models.items()}
    model_to_brands = (
        bm.groupby("model")["brand"].apply(lambda s: list(dict.fromkeys(s.tolist()))).to_dict()
    )

    # DATASET base
    df = train.copy()
    if "Brand" not in df.columns or "model" not in df.columns:
        raise KeyError("Esperava colunas 'Brand' e 'model' no train.")
    df["brand_new"] = df["Brand"].map(nullify)
    df["model_new"] = df["model"].map(nullify)

    # 1) BRAND: aliases → fuzzy (se não bater limiar, fica vazio)
    df["brand_new"] = df["brand_new"].map(lambda x: BRAND_ALIASES.get(norm_text(x), x) if x!="" else "")
    fixed_brands = []
    for b in df["brand_new"]:
        if b == "":
            fixed_brands.append("")
        elif b in brands:
            fixed_brands.append(b)
        else:
            cand, sc, _ = fuzzy_best(b, brands)
            fixed_brands.append(cand if (cand and sc >= thr_brand) else "")
    df["brand_new"] = fixed_brands

    # 2) MODEL COM BRAND FIXA: tentar SEMPRE fuzzy DENTRO da brand; só se falhar é que fica vazio
    new_models = []
    for b, m in zip(df["brand_new"], df["model_new"]):
        if b == "":
            new_models.append(m)  # tratará no passo seguinte
            continue
        # normalização específica e igualdade por chave
        m_normed = brand_model_normalize(b, m)
        key = norm_key(m_normed)
        allowed_map = brand_to_models_keymap.get(b, {})
        if key and key in allowed_map:
            new_models.append(allowed_map[key]); continue
        # fuzzy RESTRITO aos modelos dessa brand
        candidates = brand_to_models.get(b, [])
        cand, sc, _ = fuzzy_best(m_normed, candidates)
        if cand is not None and sc >= thr_model_in_brand:
            new_models.append(cand)
        else:
            new_models.append("")  # falhou fuzzy -> deixa vazio (vamos preencher mais tarde pelo mais comum)
    df["model_new"] = new_models

    # 3) BRAND vazia mas MODEL “certo e ÚNICO” → preencher brand
    filled_brands = []
    for b, m in zip(df["brand_new"], df["model_new"]):
        if b != "":
            filled_brands.append(b); continue
        if m == "":
            filled_brands.append(""); continue
        # tentar casar ao master por chave; senão fuzzy global forte
        m_can = None
        mk = norm_key(m)
        for mm in models:
            if norm_key(mm) == mk:
                m_can = mm; break
        if m_can is None:
            cand, sc, _ = fuzzy_best(m, models)
            if cand and sc >= thr_model_unique:
                m_can = cand
        if m_can:
            brands_for_m = model_to_brands.get(m_can, [])
            filled_brands.append(brands_for_m[0] if len(brands_for_m)==1 else "")
        else:
            filled_brands.append("")
    df["brand_new"] = filled_brands

    # 4) MODEL vazio/errado mas BRAND certa → usar modelo MAIS COMUM dessa brand no TEU dataset
    valid_pairs = df[(df["brand_new"]!="") & (df["model_new"]!="")]
    brand_top_model_ds = {}
    if len(valid_pairs):
        vc = valid_pairs.value_counts(["brand_new","model_new"]).reset_index(name="n")
        for b in vc["brand_new"].unique():
            sub = vc[vc["brand_new"]==b].sort_values("n", ascending=False)
            brand_top_model_ds[b] = sub.iloc[0]["model_new"]
    need_fill = (df["brand_new"]!="") & (df["model_new"]=="")
    df.loc[need_fill, "model_new"] = df.loc[need_fill, "brand_new"].map(brand_top_model_ds).fillna("")

    # 5) Ambos vazios → par (brand, model) mais comum (dataset → master)
    valid_pairs = df[(df["brand_new"]!="") & (df["model_new"]!="")]
    if len(valid_pairs):
        common_pair = valid_pairs.value_counts(["brand_new","model_new"]).idxmax()
    else:
        common_pair = bm.groupby(["brand","model"]).size().sort_values(ascending=False).index[0]
    mask_both = (df["brand_new"]=="") & (df["model_new"]=="")
    df.loc[mask_both, ["brand_new","model_new"]] = common_pair

    return df

# --- exemplo ---
df_new = clean(train, brand_model_dic)
# df_new.to_csv("clean.csv", index=False)


In [276]:
df_new.to_csv("clean.csv", index=False)


In [59]:
print(df["model"].value_counts())

model
focus       2865
c class     2159
fiesta      1929
golf        1410
corsa       1007
            ... 
terracan       1
rapi           1
k              1
camr           1
cascada        1
Name: count, Length: 269, dtype: int64


### year

In [60]:
df["year"].unique()

array([2022.87800554, 2017.        , 2016.        , 2019.        ,
       2018.        , 2011.        , 2015.        ,           nan,
       2023.26798867, 2014.        , 2020.        , 2013.        ,
       2010.        , 2024.12175905, 2008.        , 2012.        ,
       2010.56500919, 2010.37154646, 2011.21085349, 2009.        ,
       2022.69668507, 2007.        , 2023.38982198, 2023.97731126,
       2023.36707842, 2004.        , 2010.26863473, 2010.67696784,
       2023.1169636 , 2002.        , 2023.60527574, 2012.69574039,
       2006.        , 2000.        , 2003.        , 1997.        ,
       2011.11118842, 2009.81675711, 2010.7464032 , 2001.        ,
       1999.        , 1998.        , 2005.        , 1991.        ,
       1996.        ])

In [61]:
#por agora nulos ficom com 9999
df["year"] = df["year"].fillna(9999)

# Ficar só com a parte inteira e converter para int
df["year"] = df["year"].astype(float).astype(int)

print(df["year"].unique())
print(df.dtypes)

[2022 2017 2016 2019 2018 2011 2015 9999 2023 2014 2020 2013 2010 2024
 2008 2012 2009 2007 2004 2002 2006 2000 2003 1997 2001 1999 1998 2005
 1991 1996]
Brand              object
model              object
year                int64
transmission       object
mileage           float64
fuelType           object
tax               float64
mpg               float64
engineSize        float64
paintQuality%     float64
previousOwners    float64
hasDamage         float64
dtype: object


### transmission       

In [62]:
df["transmission"].unique()

array(['Automatic', 'Semi-Auto', 'Manual', 'unknow', 'Manua', 'automatic',
       nan, 'semi-auto', 'MANUAL', 'Semi-Aut', 'unknown', 'emi-Auto',
       'utomatic', 'SEMI-AUTO', 'anual', 'Automati', 'manual',
       'AUTOMATIC', ' Manual ', ' Manual', 'UNKNOWN', 'anua', 'AUTOMATI',
       'nknown', 'MANUA', 'Other', ' MANUAL ', 'manual ', 'manua',
       'UTOMATIC', 'automati', 'utomati', 'ANUAL', 'emi-auto', 'EMI-AUTO',
       'SEMI-AUT', 'Manual ', ' manual ', 'emi-Aut'], dtype=object)

In [63]:
df["transmission"] = df["transmission"].str.lower().str.strip()
df["transmission"].unique()

array(['automatic', 'semi-auto', 'manual', 'unknow', 'manua', nan,
       'semi-aut', 'unknown', 'emi-auto', 'utomatic', 'anual', 'automati',
       'anua', 'nknown', 'other', 'utomati', 'emi-aut'], dtype=object)

In [64]:
df["transmission"] = df["transmission"].str.lower().str.strip()

# Criar mapeamento só para marcas que já vi no teu print
trans_map = {
    "automatic": "Automatic", "utomatic": "Automatic", "automati": "Automatic","utomati": "Automatic",
    "semi-auto": "Semi-Auto", "semi-aut": "Semi-Auto","emi-auto": "Semi-Auto", "emi-aut": "Semi-Auto",
    "manual": "Manual", "manua": "Manual","anual": "Manual", "anua": "Manual",
    "unknow": "Unknown", "unknown": "Unknown", "nknown": "Unknown",
    "other": "Other"
}

# Substituir se existir no dicionário
df["transmission"] = df["transmission"].map(lambda x: trans_map.get(x, x))

# Ver marcas únicas já limpas
print(df["transmission"].unique())


['Automatic' 'Semi-Auto' 'Manual' 'Unknown' nan 'Other']


### mileage

In [65]:
df["mileage"].unique()
#tem de ser numero e tem de ser poitivo

array([ 30700.        , -48190.65567291,  36792.        , ...,
        27575.        ,   8297.        ,  11071.        ])

### fuelType

In [66]:
df["fuelType"].unique()

array(['petrol', 'Petrol', 'Diesel', 'Diese', 'Hybrid', 'iesel', 'Petro',
       'hybrid', 'etrol', 'DIESEL', nan, 'PETROL', 'diesel', 'Other',
       'iese', 'diese', 'etro', 'petro', 'HYBRID', 'Hybri', 'ther',
       'ETROL', 'ybrid', 'IESEL', 'DIESE', 'PETRO', 'hybri', 'other',
       'Othe', 'Electric'], dtype=object)

In [67]:
# Normalizar (minúsculas + remover espaços)
df["fuelType"] = df["fuelType"].str.lower().str.strip()

# Mapeamento só com o que foi encontrado
fuel_map = {
    # Petrol
    "petrol": "Petrol", "petro": "Petrol", "etro": "Petrol","etrol": "Petrol",
    
    # Diesel
    "diesel": "Diesel", "diese": "Diesel", "iesel": "Diesel","iese": "Diesel",
    
    # Hybrid
    "hybrid": "Hybrid", "hybri": "Hybrid","ybrid": "Hybrid",
    
    # Electric
    "electric": "Electric",
    
    # Other
    "other": "Other", "othe": "Other", "ther": "Other"
}

# Aplicar mapeamento
df["fuelType"] = df["fuelType"].map(lambda x: fuel_map.get(x, x))

# Substituir NaN por "Unknown"
df["fuelType"] = df["fuelType"].fillna("Unknown")

# Ver categorias finais
print(df["fuelType"].unique())


['Petrol' 'Diesel' 'Hybrid' 'Unknown' 'Other' 'Electric']


### tax

### mpg

### engineSize

### paintQuality%

In [68]:
df["paintQuality%"].unique()
#decidir o que fazer com os superior a 100. 1) truncar para 100. 2) apagar e preencher com a mediana 3)apagar e knn

array([ 61.        ,  60.        ,  94.        ,  77.        ,
        45.        ,  64.        ,  50.        ,  71.        ,
        99.        ,  73.        ,  78.        ,  40.        ,
        35.        ,  76.        ,  53.        ,  90.        ,
                nan,  42.        ,  70.        ,  75.        ,
        79.        ,  98.        ,  62.        ,  88.        ,
        59.        ,  97.        ,  85.        ,  89.        ,
        92.        ,  63.        ,  36.        ,  56.        ,
        54.        ,  52.        ,  66.        ,  80.        ,
        91.        ,  31.        ,  37.        ,  67.        ,
        30.        ,  84.        ,  69.        ,  55.        ,
        57.        ,  48.        ,  44.        ,  39.        ,
        82.        ,  34.        ,  33.        , 125.10995148,
         3.20741784,  65.        ,  86.        ,  74.        ,
       125.00377307,  87.        ,  72.        ,  93.        ,
        46.        ,  38.        ,  47.        ,  49.  

### previousOwners

In [69]:
df["previousOwners"].unique()

array([ 3.        ,  2.        ,  1.        ,  4.        ,  0.        ,
               nan, -2.34565   ,  6.25823052,  6.23017958,  6.25837066,
       -2.29943868,  6.21772443, -2.33512284,  6.2482512 ,  6.23308217,
       -2.33744529,  6.24583495,  6.24177863, -2.33936045, -2.31225953,
       -2.34030622, -2.31733109, -2.34010209,  6.22789796])

In [70]:
# Substituir NaN por 9999 (ou outro valor que prefiras, ex: 0)
df["previousOwners"] = df["previousOwners"].fillna(9999)

# Converter para inteiro
df["previousOwners"] = df["previousOwners"].astype(float).astype(int)

# Tornar valores negativos em positivos
df["previousOwners"] = df["previousOwners"].abs()

print(df["previousOwners"].unique())
print(df.dtypes)


[   3    2    1    4    0 9999    6]
Brand              object
model              object
year                int64
transmission       object
mileage           float64
fuelType           object
tax               float64
mpg               float64
engineSize        float64
paintQuality%     float64
previousOwners      int64
hasDamage         float64
dtype: object


### hasDamage

In [71]:
df["hasDamage"].unique()
#para apagar a variavel que nao tem nada de jeito

array([ 0., nan])

In [72]:
train["hasDamage"].unique()

array([ 0., nan])

In [73]:
# fazer exploração da media 

In [74]:
train_0 = train[train["hasDamage"]== 0]
train_nan = train[~(train["hasDamage"] == 0)]


In [75]:
train_0["price"].median()

14698.0

In [76]:
train_nan["price"].median()

14798.0

In [77]:
#como nao houve diferença na media então tiramos a variavel